In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV
import joblib
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_selection import RFE
import matplotlib.pyplot as pyplot
from sklearn.metrics import roc_curve, auc
from sklearn.feature_selection import VarianceThreshold
from pandas.api.types import is_numeric_dtype, is_categorical_dtype, is_string_dtype
from datapreparation import *

In [8]:
flights = pd.read_csv('/Users/user/Documents/Github/flightstatus/data/dev/flights_processed.csv')

In [15]:
flights.columns

Index(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'FLIGHT_NUMBER',
       'SCHEDULED_DEPARTURE', 'SCHEDULED_TIME', 'DISTANCE',
       'SCHEDULED_ARRIVAL', 'LATITUDE_ORIGIN', 'LONGITUDE_ORIGIN',
       'LATITUDE_DESTINATION', 'LONGITUDE_DESTINATION', 'TARGET',
       'HOLIDAY_FLAG', 'WEEKEND', 'DATE', 'DATE_TRAFFIC', 'MONTH_TRAFFIC',
       'AIRLINE_TRAFFIC', 'ORIGIN_AIRPORT_TRAFFIC',
       'DESTINATION_AIRPORT_TRAFFIC', 'AIRPORT_ORIGIN_INTL',
       'AIRPORT_DESTINATION_INTL', 'LONGITUDE_DIF', 'LATITUDE_DIF',
       'AIRLINE_CAT', 'TAIL_NUMBER_CAT', 'ORIGIN_AIRPORT_CAT',
       'DESTINATION_AIRPORT_CAT', 'CITY_ORIGIN_CAT', 'STATE_ORIGIN_CAT',
       'COUNTRY_ORIGIN_CAT', 'CITY_DESTINATION_CAT', 'STATE_DESTINATION_CAT',
       'COUNTRY_DESTINATION_CAT', 'HOLIDAY_CAT'],
      dtype='object')

In [16]:
flights[flights.ORIGIN_AIRPORT_CAT.isnull()]

,YEAR,MONTH,DAY,DAY_OF_WEEK,FLIGHT_NUMBER,SCHEDULED_DEPARTURE,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,LATITUDE_ORIGIN,...,TAIL_NUMBER_CAT,ORIGIN_AIRPORT_CAT,DESTINATION_AIRPORT_CAT,CITY_ORIGIN_CAT,STATE_ORIGIN_CAT,COUNTRY_ORIGIN_CAT,CITY_DESTINATION_CAT,STATE_DESTINATION_CAT,COUNTRY_DESTINATION_CAT,HOLIDAY_CAT


In [9]:
NUM_VAR = [v for v in data.columns if is_numeric_dtype(flights[v])]
STR_VAR = [v for v in data.columns if is_string_dtype(flights[v])]

In [10]:
NULL_VAR = get_col_with_null(flights)
NUM_VAR_COMPLETE = [v for v in flights.columns if is_numeric_dtype(flights[v]) and v not in [NULL_VAR]]

In [12]:
NULL_VAR

['SCHEDULED_TIME',
 'LATITUDE_ORIGIN',
 'LONGITUDE_ORIGIN',
 'LATITUDE_DESTINATION',
 'LONGITUDE_DESTINATION',
 'LONGITUDE_DIF',
 'LATITUDE_DIF']

In [14]:
flights[flights.LATITUDE_ORIGIN.isnull()]

,YEAR,MONTH,DAY,DAY_OF_WEEK,FLIGHT_NUMBER,SCHEDULED_DEPARTURE,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,LATITUDE_ORIGIN,...,TAIL_NUMBER_CAT,ORIGIN_AIRPORT_CAT,DESTINATION_AIRPORT_CAT,CITY_ORIGIN_CAT,STATE_ORIGIN_CAT,COUNTRY_ORIGIN_CAT,CITY_DESTINATION_CAT,STATE_DESTINATION_CAT,COUNTRY_DESTINATION_CAT,HOLIDAY_CAT
2098,2015,1,1,4,1652,835,80.0,401,955,NaN,...,731,97,38,224,10,1,205,45,1,8
3291,2015,1,1,4,5338,959,61.0,240,1200,NaN,...,192,97,21,224,10,1,19,11,1,8
5504,2015,1,1,4,4207,1220,120.0,798,1520,NaN,...,2565,97,52,224,10,1,24,22,1,8
6900,2015,1,1,4,1743,1349,71.0,240,1600,NaN,...,4762,97,21,224,10,1,19,11,1,8
7624,2015,1,1,4,263,1435,115.0,571,1630,NaN,...,3617,97,144,224,10,1,132,46,1,8
10444,2015,1,1,4,1958,1735,75.0,240,1950,NaN,...,4599,97,21,224,10,1,19,11,1,8
10875,2015,1,1,4,1359,1800,115.0,658,1955,NaN,...,593,314,150,278,10,1,52,48,1,8
14435,2015,1,2,5,508,600,68.0,240,808,NaN,...,4361,97,21,224,10,1,19,11,1,0
17343,2015,1,2,5,1652,835,80.0,401,955,NaN,...,3207,97,38,224,10,1,205,45,1,0
18719,2015,1,2,5,5338,959,61.0,240,1200,NaN,...,475,97,21,224,10,1,19,11,1,0


In [11]:
selector = VarianceThreshold()
selector.fit(flights[NUM_VAR_COMPLETE])
# NUM_VAR_REMOVED = [NUM_VAR_COMPLETE[i] for i in range(len(NUM_VAR_COMPLETE)) if selector.get_support()[i] == False]

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [68]:
df_train, df_test = train_test_split(data, stratify = data['TARGET'], test_size=0.3, random_state=12)

In [69]:
selector = VarianceThreshold()
selector.fit(df_train[NUM_VAR])

VarianceThreshold(threshold=0.0)

In [72]:
NUM_VAR_NOTSELECTED = [NUM_VAR[i] for i in range(len(NUM_VAR)) if selector.get_support()[i] ==False]
NUM_VAR_NOTSELECTED

['YEAR',
 'MONTH',
 'DAY',
 'DAY_OF_WEEK',
 'HOLIDAY_FLAG',
 'WEEKEND',
 'DATE_TRAFFIC',
 'MONTH_TRAFFIC']

In [70]:
NUM_VAR_SELECTED = [NUM_VAR[i] for i in range(len(NUM_VAR)) if selector.get_support()[i]]
NUM_VAR_SELECTED

['FLIGHT_NUMBER',
 'SCHEDULED_DEPARTURE',
 'SCHEDULED_TIME',
 'DISTANCE',
 'SCHEDULED_ARRIVAL',
 'LATITUDE_ORIGIN',
 'LONGITUDE_ORIGIN',
 'LATITUDE_DESTINATION',
 'LONGITUDE_DESTINATION',
 'TARGET',
 'AIRLINE_CAT',
 'ORIGIN_AIRPORT_CAT',
 'DESTINATION_AIRPORT_CAT',
 'AIRLINE_TRAFFIC',
 'ORIGIN_AIRPORT_TRAFFIC',
 'DESTINATION_AIRPORT_TRAFFIC',
 'AIRPORT_ORIGIN_INTL',
 'AIRPORT_DESTINATION_INTL',
 'LONGITUDE_DIF',
 'LATITUDE_DIF']

In [18]:
NUM_VAR

['YEAR',
 'MONTH',
 'DAY',
 'DAY_OF_WEEK',
 'FLIGHT_NUMBER',
 'SCHEDULED_DEPARTURE',
 'DEPARTURE_TIME',
 'DEPARTURE_DELAY',
 'TAXI_OUT',
 'WHEELS_OFF',
 'SCHEDULED_TIME',
 'ELAPSED_TIME',
 'AIR_TIME',
 'DISTANCE',
 'WHEELS_ON',
 'TAXI_IN',
 'SCHEDULED_ARRIVAL',
 'ARRIVAL_TIME',
 'ARRIVAL_DELAY',
 'DIVERTED',
 'CANCELLED',
 'AIR_SYSTEM_DELAY',
 'SECURITY_DELAY',
 'AIRLINE_DELAY',
 'LATE_AIRCRAFT_DELAY',
 'WEATHER_DELAY',
 'LATITUDE_ORIGIN',
 'LONGITUDE_ORIGIN',
 'LATITUDE_DESTINATION',
 'LONGITUDE_DESTINATION',
 'TARGET',
 'AIRLINE_CAT',
 'ORIGIN_AIRPORT_CAT',
 'DESTINATION_AIRPORT_CAT',
 'HOLIDAY_FLAG',
 'WEEKEND',
 'DATE_TRAFFIC',
 'MONTH_TRAFFIC',
 'AIRLINE_TRAFFIC',
 'ORIGIN_AIRPORT_INTL',
 'ORIGIN_AIRPORT_TRAFFIC',
 'DESTINATION_AIRPORT_INTL',
 'DESTINATION_AIRPORT_TRAFFIC',
 'LONGITUDE_DIF',
 'LATITUDE_DIF']

In [81]:
nulldf = missing_percent(data, data.columns)

The percentage of missing data


In [86]:
nulldf = data.isnull().sum()

In [89]:
nulldf[nulldf>0].index.tolist()

['TAIL_NUMBER']